In [1]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized")

/Users/oliverklingefjord/dev/meaning-alignment/mgg/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Print out distribution of lengths to "chosen" in training data
lengths = [len(x["chosen"]) for x in ds["train_prefs"]]

In [9]:
min(lengths), max(lengths)

(2, 2)

In [10]:
from datasets import load_dataset

dataset = load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")

In [11]:
lengths = [len(x["chosen"]) for x in ds["train_prefs"]]

min(lengths), max(lengths)

(2, 2)

In [17]:
!pip install sentencepiece

  Using cached sentencepiece-0.2.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.7 kB)
Using cached sentencepiece-0.2.0-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)


In [18]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load Tokenizer from the hub
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b"  # replace with your model id
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load dataset from the hub
dataset = load_dataset(
    "argilla/ultrafeedback-binarized-preferences-cleaned", split="train"
)
dataset = dataset.shuffle().select(range(13750))


def rec_extract_assistant_messages(messages, index=-1):
    """Recursively extract the last assistant messages from the end of the conversation."""
    if messages[index]["role"] == "assistant":
        return [messages[index]]
    else:
        return rec_extract_assistant_messages(messages, index - 1)


# System message used if there is no system message at the beginning of the conversation
# Can be repelaced and modified as needed
DEFAULT_SYSTEM_MESSAGE = "You are Dolphin, a helpful AI assistant."


def create_triplets(example, tokenizer, default_system_message=DEFAULT_SYSTEM_MESSAGE):
    """Create the triplets (prompt, chosen, rejected)"""
    # Extract the N-1 turns to form the prompt
    # Prepend a system message if the first message is not a system message
    prompt_messages = example["chosen"][:-1]
    if example["chosen"][0]["role"] != "system":
        prompt_messages.insert(0, {"role": "system", "content": default_system_message})
    # Now we extract the final assistant turn to define chosen/rejected responses
    chosen_messages = rec_extract_assistant_messages(example["chosen"])
    rejected_messages = rec_extract_assistant_messages(example["rejected"])

    # apply template to the messages and return the triplets
    return {
        "prompt": tokenizer.apply_chat_template(prompt_messages, tokenize=False),
        "chosen": tokenizer.apply_chat_template(chosen_messages, tokenize=False),
        "rejected": tokenizer.apply_chat_template(rejected_messages, tokenize=False),
    }


dataset = dataset.map(
    create_triplets, remove_columns=dataset.features, fn_kwargs={"tokenizer": tokenizer}
)
# split dataset into 11,000 training samples and 2,750 test samples
dataset = dataset.train_test_split(test_size=2750 / 13750)

# print sample cut of
print(dataset["train"][0]["prompt"][:50])
print(dataset["train"][0]["chosen"][:50])
print(dataset["train"][0]["rejected"][:50])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [19]:
from datasets import load_dataset
from transformers import AutoTokenizer

ds = load_dataset("argilla/OpenHermesPreferences", split="train")

# Load a tokenizer and apply chat template
tokenizer = AutoTokenizer.from_pretrained("teknium/OpenHermes-2.5-Mistral-7B")
example = ds[0]
chosen_example = tokenizer.apply_chat_template(example["chosen"], tokenize=False)
rejected_example = tokenizer.apply_chat_template(example["rejected"], tokenize=False)
print(f"== Chosen example ==\n\n{chosen_example}")
print(f"== Rejected example ==\n\n{rejected_example}")